# 第10章: ベクトル空間法 (II)

第10章では，前章に引き続き単語ベクトルの学習に取り組む．

In [1]:
from gensim.models import Word2Vec

# 90. word2vecによる学習
81で作成したコーパスに対してword2vecを適用し，単語ベクトルを学習せよ．さらに，学習した単語ベクトルの形式を変換し，86-89のプログラムを動かせ．

In [4]:
# https://rare-technologies.com/word2vec-tutorial/
with open('81.txt', 'r') as f:
    corpus = [f.read().split()]
model = Word2Vec(corpus, min_count=1)    

In [8]:
# 86 ベクトル表示
model['United_States']

array([ -2.74452008e-03,   4.14247252e-03,  -1.90847181e-03,
         4.92690550e-03,  -1.66915284e-04,  -2.07025977e-03,
        -4.08928172e-04,   3.09598702e-03,   2.16265267e-04,
        -4.32988349e-03,   2.14363984e-03,  -3.80866259e-04,
         3.33504495e-03,  -4.16176999e-03,  -1.51679886e-03,
         2.35196599e-03,   1.07113540e-03,   4.17051185e-03,
        -4.32816334e-03,  -3.14497110e-03,   9.37194913e-04,
        -7.68948055e-04,  -3.61964339e-03,   7.99834612e-04,
         4.82065696e-03,  -9.44846310e-04,  -2.27891942e-04,
         1.80317799e-03,  -4.52356879e-03,   1.69966195e-03,
        -5.80497459e-03,  -5.35298139e-03,   3.81616876e-03,
         4.19021538e-03,  -1.97405810e-03,   5.51197631e-03,
        -3.72335850e-03,  -1.29071646e-03,  -1.52754062e-03,
        -5.11130644e-03,   3.57222394e-03,   1.73195603e-03,
        -4.50111739e-03,   1.14969956e-03,  -4.23314143e-03,
         2.27533001e-03,   6.00323407e-03,  -2.74500740e-03,
         3.50066344e-04,

In [9]:
# 87 コサイン類似度 
model.similarity("United_States", "U.S")

0.18035988021980459

In [12]:
# 88 類似度 top10
model.most_similar(positive=['England'], topn=10)

[('Wisconsin-Madison', 0.44466376304626465),
 ('audio-only', 0.44343793392181396),
 ('reentry', 0.4241366386413574),
 ('Lille', 0.41284117102622986),
 ('nasugdan', 0.40986520051956177),
 ('celibate', 0.4096040725708008),
 ('Touane', 0.4047726094722748),
 ('16-member', 0.400925874710083),
 ('wrestling', 0.4006977677345276),
 ("Honey's", 0.3988679051399231)]

In [10]:
# 89 アナロジー top10
model.most_similar(positive=['Spain', 'Athens'], negative=['Madrid'], topn=10)

[('imagine.', 0.45201873779296875),
 ('হিমু', 0.4420362114906311),
 ('Glinka)', 0.42740336060523987),
 ('Pacifique', 0.40670719742774963),
 ('20,000–30,000', 0.40599608421325684),
 ('Brutaal', 0.40582478046417236),
 ('Fuzhounese', 0.40473905205726624),
 ('bap', 0.4021039605140686),
 ('inquiry.', 0.4017986059188843),
 ('Krautfabrik', 0.40117621421813965)]

# 91. アナロジーデータの準備
単語アナロジーの評価データをダウンロードせよ．このデータ中で": "で始まる行はセクション名を表す．例えば，": capital-common-countries"という行は，"capital-common-countries"というセクションの開始を表している．ダウンロードした評価データの中で，"family"というセクションに含まれる評価事例を抜き出してファイルに保存せよ．

In [13]:
with open('questions-words.txt', 'r') as f:
    sections = f.read().split(':')
    for section in sections:
        if 'family' in section.split('\n')[0]:
            family = "\n".join(section.split('\n')[1:])
with open('family.txt', 'w') as f:
    f.write(family)

# 92. アナロジーデータへの適用
91で作成した評価データの各事例に対して，vec(2列目の単語) - vec(1列目の単語) + vec(3列目の単語)を計算し，そのベクトルと類似度が最も高い単語と，その類似度を求めよ．求めた単語と類似度は，各事例の末尾に追記せよ．このプログラムを85で作成した単語ベクトル，90で作成した単語ベクトルに対して適用せよ．

In [16]:
with open('family.txt', 'r') as f:
    family = [x[:-1].split() for x in f.readlines()]
len(family)

506

In [39]:
def check_contain(words):
    for w in words:
        if not w in model:
            return False 
    return True

In [41]:
for line in family:
    c1, c2, c3, c4 = line
    nomatch = model.doesnt_match(line)
    if check_contain(line):
        w, p = model.most_similar(positive=[c2, c3], negative=[c1], topn=1)[0]
        print(w, p)
    else:
        print("NoMatch", 0)

dusk" 0.4291321039199829
1984–1988) 0.42714399099349976
Poules) 0.44175904989242554
EMLL 0.4708064794540405
Bhikkhus 0.4463440775871277
NoMatch 0
snowmobiles 0.4089697003364563
sheeting 0.4304097294807434
Altbayern 0.41197681427001953
Altbayern 0.4615488648414612
Janáček 0.4465686082839966
Hocking 0.4841853976249695
gridirons 0.44049376249313354
Pooja 0.4197467267513275
Enix's 0.4253019094467163
snowmobiles 0.43248608708381653
Polynesia 0.42437052726745605
Airliners 0.4742032587528229
FRAS 0.4796660542488098
Opperman 0.46542245149612427
“Measurements 0.4417111873626709
brown-throated 0.46532654762268066
Tribune-Herald 0.4832778871059418
civil" 0.4388255476951599
Autonome 0.40952053666114807
sadomasochism" 0.4658799469470978
NoMatch 0
GOES-West 0.42592501640319824
Intelligencer 0.4576433300971985
Putting 0.4567410945892334
Michelin 0.4614273011684418
ION 0.45787280797958374
inches" 0.42312508821487427
pretenders 0.43978092074394226
Cebuano's 0.48285239934921265
Quebecor 0.42304998636245

# 93. アナロジータスクの正解率の計算
92で作ったデータを用い，各モデルのアナロジータスクの正解率を求めよ．

# 94. WordSimilarity-353での類似度計算
The WordSimilarity-353 Test Collectionの評価データを入力とし，1列目と2列目の単語の類似度を計算し，各行の末尾に類似度の値を追加するプログラムを作成せよ．このプログラムを85で作成した単語ベクトル，90で作成した単語ベクトルに対して適用せよ．

# 95. WordSimilarity-353での評価
94で作ったデータを用い，各モデルが出力する類似度のランキングと，人間の類似度判定のランキングの間のスピアマン相関係数を計算せよ．

# 96. 国名に関するベクトルの抽出
word2vecの学習結果から，国名に関するベクトルのみを抜き出せ．

# 97. k-meansクラスタリング
96の単語ベクトルに対して，k-meansクラスタリングをクラスタ数k=5k=5として実行せよ．

# 98. Ward法によるクラスタリング
96の単語ベクトルに対して，Ward法による階層型クラスタリングを実行せよ．さらに，クラスタリング結果をデンドログラムとして可視化せよ．

# 99. t-SNEによる可視化
96の単語ベクトルに対して，ベクトル空間をt-SNEで可視化せよ．